In [1]:
pip install pandas numpy scipy yfinance quantstats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import yfinance as yf 
import pandas as pd 
from datetime import datetime, timedelta

In [6]:
TICKER_SYMBOL = 'SPY'
START_DATE = '2023-01-01'
END_DATE = '2024-12-31'
OUTPUT_CSV_FILE = f'spy_options_historical_{START_DATE}_to_{END_DATE}.csv'

In [26]:
def get_options_data_for_day(TICKER_SYMBOL, START_DATE) :
    ticker = yf.Ticker(TICKER_SYMBOL)
    hist = ticker.history(start= START_DATE , end= START_DATE +timedelta(days=1))
    if hist.empty:
        return None 

    underlying_price = hist['Close'].iloc[0]
    quote_date = hist.index[0].normalize()

    try: 
        expirations = ticker.options
    except IndexError: 
        return None

    all_options_list = []
    for expiry in expirations:
        try:
            opt_chain = ticker.option_chain(expiry)
            calls = opt_chain.calls
            calls['option_type'] = 'C'
            puts = opt_chain.puts
            puts['option_type'] = 'P'
            all_options_list.extend([calls, puts])
        except (IndexError, ValueError, TypeError): # Handle various errors yfinance can throw
            continue
            
    if not all_options_list:
        return None
        
    full_chain_df = pd.concat(all_options_list)
    full_chain_df['quote_date'] = quote_date
    full_chain_df['underlying_price'] = underlying_price
    full_chain_df['expiration'] = pd.to_datetime(full_chain_df['contractSymbol'].str[len(TICKER_SYMBOL):len(TICKER_SYMBOL)+6], format='%y%m%d')
    
    required_columns = ['quote_date', 'expiration', 'strike', 'option_type', 'bid', 'ask', 'underlying_price']
    return full_chain_df[required_columns]


def main():
    print("Starting historical data acquisition...")
    master_df = []
    
    # Create a range of business days for the backtest period
    date_range = pd.bdate_range(start=START_DATE, end=END_DATE)
    
    for date in date_range:
        print(f"Fetching data for: {date.strftime('%Y-%m-%d')}")
        try:
            daily_data = get_options_data_for_day(TICKER_SYMBOL, date)
            if daily_data is not None and not daily_data.empty:
                master_df.append(daily_data)
            else:
                print(f"  -> No data found for {date.strftime('%Y-%m-%d')}.")
        except Exception as e:
            print(f"  -> An error occurred for {date.strftime('%Y-%m-%d')}: {e}")

    if not master_df:
        print("\nNo data could be downloaded. Exiting.")
        return

    print("\nCombining all data into a single DataFrame...")
    final_df = pd.concat(master_df)
    
    # Clean final data
    final_df = final_df[(final_df['bid'] > 0) & (final_df['ask'] > 0)].copy()
    
    print(f"Saving data to {OUTPUT_CSV_FILE}...")
    final_df.to_csv(OUTPUT_CSV_FILE, index=False)
    print("Data preparation complete!")

if __name__ == "__main__":
    main()

Starting historical data acquisition...
Fetching data for: 2023-01-02


$SPY: possibly delisted; no price data found  (1d 2023-01-02 00:00:00 -> 2023-01-03 00:00:00)


  -> No data found for 2023-01-02.
Fetching data for: 2023-01-03
Fetching data for: 2023-01-04
Fetching data for: 2023-01-05
Fetching data for: 2023-01-06
Fetching data for: 2023-01-09
Fetching data for: 2023-01-10
Fetching data for: 2023-01-11
Fetching data for: 2023-01-12
Fetching data for: 2023-01-13


$SPY: possibly delisted; no price data found  (1d 2023-01-16 00:00:00 -> 2023-01-17 00:00:00)


Fetching data for: 2023-01-16
  -> No data found for 2023-01-16.
Fetching data for: 2023-01-17
Fetching data for: 2023-01-18
Fetching data for: 2023-01-19
Fetching data for: 2023-01-20
Fetching data for: 2023-01-23
Fetching data for: 2023-01-24
Fetching data for: 2023-01-25
Fetching data for: 2023-01-26
Fetching data for: 2023-01-27
Fetching data for: 2023-01-30
Fetching data for: 2023-01-31
Fetching data for: 2023-02-01
Fetching data for: 2023-02-02
Fetching data for: 2023-02-03
Fetching data for: 2023-02-06
Fetching data for: 2023-02-07
Fetching data for: 2023-02-08
Fetching data for: 2023-02-09
Fetching data for: 2023-02-10
Fetching data for: 2023-02-13
Fetching data for: 2023-02-14
Fetching data for: 2023-02-15
Fetching data for: 2023-02-16
Fetching data for: 2023-02-17


...

Fetching data for: 2024-09-02
  -> No data found for 2024-09-02.
Fetching data for: 2024-09-03
Fetching data for: 2024-09-04
Fetching data for: 2024-09-05
Fetching data for: 2024-09-06
Fetching data for: 2024-09-09
Fetching data for: 2024-09-10
Fetching data for: 2024-09-11
Fetching data for: 2024-09-12
Fetching data for: 2024-09-13
Fetching data for: 2024-09-16
Fetching data for: 2024-09-17
Fetching data for: 2024-09-18
Fetching data for: 2024-09-19
Fetching data for: 2024-09-20
Fetching data for: 2024-09-23
Fetching data for: 2024-09-24
Fetching data for: 2024-09-25
Fetching data for: 2024-09-26
Fetching data for: 2024-09-27
Fetching data for: 2024-09-30
Fetching data for: 2024-10-01
Fetching data for: 2024-10-02
Fetching data for: 2024-10-03
Fetching data for: 2024-10-04
Fetching data for: 2024-10-07
Fetching data for: 2024-10-08
Fetching data for: 2024-10-09
Fetching data for: 2024-10-10
Fetching data for: 2024-10-11
Fetching data for: 2024-10-14
Fetching data for: 2024-10-15
Fetch

$SPY: possibly delisted; no price data found  (1d 2024-11-28 00:00:00 -> 2024-11-29 00:00:00)


Fetching data for: 2024-11-28
  -> No data found for 2024-11-28.
Fetching data for: 2024-11-29
Fetching data for: 2024-12-02
Fetching data for: 2024-12-03
Fetching data for: 2024-12-04
Fetching data for: 2024-12-05
Fetching data for: 2024-12-06
Fetching data for: 2024-12-09
Fetching data for: 2024-12-10
Fetching data for: 2024-12-11
Fetching data for: 2024-12-12
Fetching data for: 2024-12-13
Fetching data for: 2024-12-16
Fetching data for: 2024-12-17
Fetching data for: 2024-12-18
Fetching data for: 2024-12-19
Fetching data for: 2024-12-20
Fetching data for: 2024-12-23
Fetching data for: 2024-12-24


$SPY: possibly delisted; no price data found  (1d 2024-12-25 00:00:00 -> 2024-12-26 00:00:00)


Fetching data for: 2024-12-25
  -> No data found for 2024-12-25.
Fetching data for: 2024-12-26
Fetching data for: 2024-12-27
Fetching data for: 2024-12-30
Fetching data for: 2024-12-31

Combining all data into a single DataFrame...
Saving data to spy_options_historical_2023-01-01_to_2024-12-31.csv...
Data preparation complete!
